# 가스공급량 외 다른 데이터 활용

In [2]:
import numpy as np 
import pandas as pd 
import lightgbm as lgb

In [3]:
data = pd.read_csv('한국가스공사_시간별 공급량_20181231.csv', encoding ='cp949')
data['연월일'] = pd.to_datetime(data['연월일'])
data['year'] = data['연월일'].dt.year
data['month'] = data['연월일'].dt.month
data['day'] = data['연월일'].dt.day
data['weekday'] = data['연월일'].dt.weekday

d_map ={}
for i, d in enumerate(data['구분'].unique()):
    d_map[d] = i 
data['구분'] = data['구분'].map(d_map)


test = pd.read_csv('가스공급량 수요예측모델개발_data/test.csv')
sub = pd.read_csv('가스공급량 수요예측모델개발_data/sample_submission.csv')
test['일자'] = test['일자|시간|구분'].str.split(' ').str[0]
test['시간'] = test['일자|시간|구분'].str.split(' ').str[1].astype(int)
test['구분'] = test['일자|시간|구분'].str.split(' ').str[2]
test['일자'] = pd.to_datetime(test['일자'])
test['year'] = test['일자'].dt.year
test['month'] = test['일자'].dt.month
test['day'] = test['일자'].dt.day
test['weekday'] = test['일자'].dt.weekday
test['구분'] = test['구분'].map(d_map)

In [4]:
data

,연월일,시간,구분,공급량,year,month,day,weekday
0,2013-01-01,1,0,2497.129,2013,1,1,1
1,2013-01-01,2,0,2363.265,2013,1,1,1
2,2013-01-01,3,0,2258.505,2013,1,1,1
3,2013-01-01,4,0,2243.969,2013,1,1,1
4,2013-01-01,5,0,2344.105,2013,1,1,1
...,...,...,...,...,...,...,...,...
368083,2018-12-31,20,6,681.033,2018,12,31,0
368084,2018-12-31,21,6,669.961,2018,12,31,0
368085,2018-12-31,22,6,657.941,2018,12,31,0
368086,2018-12-31,23,6,610.953,2018,12,31,0


## 가스 공급량 외 다른 데이터 
- 한국전력거레소 시간별 발전량 : 2017년 1월 부터 2020년 12월까지 데이터   
    제공된 train 데이터중 17년 부터 활용한 모델 개발 

In [6]:
elc = pd.read_csv('한국전력거래소_시간별 발전량_20201231.csv', encoding = 'cp949')

In [7]:
elc

,거래일,1시,2시,3시,4시,5시,6시,7시,8시,9시,...,15시,16시,17시,18시,19시,20시,21시,22시,23시,24시
0,2017-01-01,55122.00025,52867.08431,51112.85964,50228.43714,50270.22378,50799.94295,50977.49163,50159.77005,49592.16798,...,49326.41341,49371.40992,49589.38287,51882.15402,53668.27648,53537.42430,53490.78492,53574.56477,54804.75240,55415.26297
1,2017-01-02,53229.02548,52920.74405,52521.67207,52558.05400,53271.30338,54905.05820,56645.02366,58570.71254,64429.09414,...,65638.65262,65560.15769,66293.47450,66666.54602,65854.39132,64004.04846,62153.75753,60619.74204,60606.06141,61807.70918
2,2017-01-03,58347.54895,57012.44028,56541.88017,56513.31631,57373.69797,58908.70117,61596.09922,64252.66753,68662.06271,...,66752.04802,66488.28873,67703.83451,68683.59288,68571.18682,66663.57764,64526.28462,62889.17741,62559.95542,63718.92409
3,2017-01-04,60016.32336,58366.48843,58643.34614,58715.11106,58916.98777,59710.07343,61895.51298,64259.83376,69180.34447,...,67257.05729,66809.35692,67653.22112,68143.51424,67470.51174,65571.37045,63998.52134,62667.07412,62512.13690,63744.05670
4,2017-01-05,59751.77799,57865.01145,58001.78231,57893.72453,58456.74216,59722.97433,62058.10242,64262.80299,69080.14776,...,69860.23713,69280.43397,69700.31720,69391.24809,68039.52590,65975.46819,63804.64394,62122.48427,61813.14500,63178.04751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1456,2020-12-27,56907.94023,55263.85119,54509.86669,54302.74277,54427.37426,55111.31336,55369.52876,54912.44000,54287.05380,...,54769.26243,55593.88808,56958.39495,59255.80044,59976.82793,59152.58702,58299.85071,57011.24959,57405.14323,57879.55667
1457,2020-12-28,55303.38221,53867.75542,53593.55773,53633.36705,54686.46798,56602.62245,60392.09869,64975.96992,71583.83459,...,66717.18025,67589.84730,70116.99232,70609.45784,69484.16264,67367.38019,65468.19805,63194.94007,62494.80122,63855.76471
1458,2020-12-29,60603.57923,58670.88951,58175.41973,58679.62442,59656.28676,61194.96948,63613.98119,67437.63509,72897.90959,...,75069.34985,75029.24404,75429.15474,74007.09861,71611.59551,69816.44913,67918.50423,65168.30364,64290.29541,65855.87134
1459,2020-12-30,63550.47923,61662.77370,60808.41559,61434.17188,62181.37960,64138.69224,67057.50739,71360.28479,77324.77648,...,77835.81539,78377.08386,79865.29939,78778.08163,76140.22272,73281.13142,70829.03643,67635.75246,66881.72669,68337.16958


In [18]:
time_elc = elc.set_index('거래일').stack().reset_index()

In [38]:
time_elc.columns = ['연월일','시간','전력발전량']

In [28]:
time_elc['시간'] = time_elc['시간'].str.replace('시', '').astype(int)

In [40]:
time_elc[:5]

,연월일,시간,전력발전량
0,2017-01-01,1,55122.00025
1,2017-01-01,2,52867.08431
2,2017-01-01,3,51112.85964
3,2017-01-01,4,50228.43714
4,2017-01-01,5,50270.22378


In [50]:
data[data['year']==2017 ]

,연월일,시간,구분,공급량,year,month,day,weekday
245448,2017-01-01,1,0,1637.137,2017,1,1,6
245449,2017-01-01,2,0,1528.425,2017,1,1,6
245450,2017-01-01,3,0,1425.081,2017,1,1,6
245451,2017-01-01,4,0,1422.889,2017,1,1,6
245452,2017-01-01,5,0,1519.809,2017,1,1,6
...,...,...,...,...,...,...,...,...
306763,2017-12-31,20,6,517.264,2017,12,31,6
306764,2017-12-31,21,6,530.896,2017,12,31,6
306765,2017-12-31,22,6,506.287,2017,12,31,6
306766,2017-12-31,23,6,470.638,2017,12,31,6


In [46]:
data['연월일']=data['연월일'].astype(str)
time_elc['연월일'] = time_elc['연월일'].astype(str)

In [51]:
new_data = pd.merge(time_elc, data, how = 'left', right_index = False)

In [56]:
train_data = new_data.dropna()

In [59]:
train_data.year = train_data.year.astype(int)
train_data.month = train_data.month.astype(int)
train_data.day = train_data.day.astype(int)
train_data.weekday = train_data.weekday.astype(int)
train_data.구분 = train_data.구분.astype(int)

/Users/openknowl/miniforge3/envs/tf_pu/lib/python3.9/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [60]:
train_data

,연월일,시간,전력발전량,구분,공급량,year,month,day,weekday
0,2017-01-01,1,55122.00025,0,1637.137,2017,1,1,6
1,2017-01-01,1,55122.00025,1,1569.005,2017,1,1,6
2,2017-01-01,1,55122.00025,2,190.978,2017,1,1,6
3,2017-01-01,1,55122.00025,3,1077.821,2017,1,1,6
4,2017-01-01,1,55122.00025,4,2160.861,2017,1,1,6
...,...,...,...,...,...,...,...,...,...
122635,2018-12-31,24,64571.97440,2,237.911,2018,12,31,0
122636,2018-12-31,24,64571.97440,3,1422.478,2018,12,31,0
122637,2018-12-31,24,64571.97440,4,3534.260,2018,12,31,0
122638,2018-12-31,24,64571.97440,5,3982.757,2018,12,31,0


In [86]:
y = train_data['공급량'] 
x = train_data[['전력발전량','구분','year','month','day','weekday']]

In [63]:
from sklearn.preprocessing import MinMaxScaler

In [65]:
scaler = MinMaxScaler(feature_range = (0,1))

In [87]:
scaler.fit(pd.DataFrame(x['전력발전량']))

MinMaxScaler()

In [88]:
x['전력발전량'] = scaler.transform(pd.DataFrame(x['전력발전량']))

/var/folders/lf/472dwzh15hqd9qhdk_3h5sx40000gp/T/ipykernel_58474/3223950431.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['전력발전량'] = scaler.transform(pd.DataFrame(x['전력발전량']))


In [90]:
from sklearn.model_selection import train_test_split

In [91]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, shuffle = True, 
                                                    random_state = 1000)

In [92]:
d_train = lgb.Dataset(X_train, y_train)
d_test = lgb.Dataset(X_test, y_test)

In [93]:
params = {'objective' : 'regression',
         'metric':'mae',
         'seed': 1000}

In [95]:
lgbr = lgb.train(params, d_train, 10000, d_test, verbose_eval = 1000,
                  early_stopping_rounds = 1000)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000857 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 317
[LightGBM] [Info] Number of data points in the train set: 91980, number of used features: 6
[LightGBM] [Info] Start training from score 990.685782
Training until validation scores don't improve for 1000 rounds
[1000]	valid_0's l1: 102.631
[2000]	valid_0's l1: 94.4157
[3000]	valid_0's l1: 90.475
[4000]	valid_0's l1: 88.3083
[5000]	valid_0's l1: 86.7703
[6000]	valid_0's l1: 85.6409
[7000]	valid_0's l1: 84.9236
[8000]	valid_0's l1: 84.3729
[9000]	valid_0's l1: 84.0025
[10000]	valid_0's l1: 83.7309
Did not meet early stopping. Best iteration is:
[9997]	valid_0's l1: 83.7284


In [102]:
test_x = test[['일자','시간','구분','year','month','day','weekday']]
test_x.columns = ['연월일','시간','구분','year','month','day','weekday']

In [105]:
test_x['연월일'] = test_x['연월일'].astype(str)

/var/folders/lf/472dwzh15hqd9qhdk_3h5sx40000gp/T/ipykernel_58474/767551699.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x['연월일'] = test_x['연월일'].astype(str)


In [109]:
test_x = pd.merge(test_x, time_elc, how = 'left', right_index = False)

In [112]:
test_x = test_x[['전력발전량','구분','year','month','day','weekday']]

In [120]:
test_x['전력발전량'] = scaler.transform(pd.DataFrame(test_x['전력발전량']))

/var/folders/lf/472dwzh15hqd9qhdk_3h5sx40000gp/T/ipykernel_58474/2781616990.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_x['전력발전량'] = scaler.transform(pd.DataFrame(test_x['전력발전량']))


In [121]:
pred = lgbr.predict(test_x)

In [122]:
sub['공급량'] = pred

In [123]:
sub

,일자|시간|구분,공급량
0,2019-01-01 01 A,2207.766842
1,2019-01-01 02 A,2204.153446
2,2019-01-01 03 A,1984.027403
3,2019-01-01 04 A,1997.392718
4,2019-01-01 05 A,2004.170234
...,...,...
15115,2019-03-31 20 H,279.563437
15116,2019-03-31 21 H,279.563437
15117,2019-03-31 22 H,306.571865
15118,2019-03-31 23 H,306.571865


In [124]:
sub.to_csv('submission/lgbm_elecData.csv', index = False)